**Draft Model for Mice Steep Stage Analysis**

In [ ]:
import random
import numpy as np
import pandas as pd
import os
import re
import copy
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
from torchvision import transforms, datasets, models

from collections import OrderedDict

import torch.utils.data as utils

from sklearn.preprocessing import LabelBinarizer

import tkinter as tk #tk for file dialog (requires Jinja2!!!)
from tkinter import filedialog #tkinter for file dialog

import re #regex for parsing
from os.path import exists


In [16]:
from inputMassager import *
inputHandler = inputMassager()

filepath = inputHandler.askForInput("Blah!")

#Period Size Variable : effects CNN architecture
periodSize = 200

#makePeriodFromTxt(self, filepath, periodSize, maxPeriods=None):
periods = makePeriodFromTxt(filepath, periodSize, 5000)


# target = regan and audreys code ---> This is what will go into the data loader
filepath2 = inputHandler.askForInput("Blah!")
timestamps = find_time_labels(filepath2)
labels, _, _ = label_dataframe_new(periods, timestamps)


Make a tensor out of eeg data (c1)

In [17]:
def getChannelTensors():
    #The tensor has to be 3 dimesional with the second dimesion equal to one in order to pass it through a 1d convolutional layer
    c1_tensor = torch.zeros((len(periods.c1.values), 1, periodSize)) 
    c2_tensor = torch.zeros((len(periods.c2.values), 1, periodSize)) 
    for i in range(len(periods.c1.values)):
        c1_tensor[i, 0] = torch.tensor(periods.c1.values[i])
        c2_tensor[i, 0] = torch.tensor(periods.c1.values[i])
    
    return c1_tensor, c2_tensor

eeg_samples, emg_samples = getChannelTensors()

Make the train and validation data loaders  

In [18]:
#print(eeg_samples)
print(torch.tensor(labels))

ds = torch.utils.data.TensorDataset(eeg_samples, torch.tensor(labels))
train_dataset, val_dataset = torch.utils.data.random_split(ds, [int(len(ds) *.80), int(len(ds) *.2)])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=100, shuffle=True)

tensor([4, 2, 2,  ..., 2, 2, 2])


/var/folders/dg/6gzrkj09311g91kn90n1qdcc0000gn/T/ipykernel_73804/29316696.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.tensor(labels))
/var/folders/dg/6gzrkj09311g91kn90n1qdcc0000gn/T/ipykernel_73804/29316696.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ds = torch.utils.data.TensorDataset(eeg_samples, torch.tensor(labels))


In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Creating class for Model**

In [53]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        # This just calls the base class constructor
        super().__init__()
        # Neural network layers assigned as attributes of a Module subclass
        # have their parameters registered for training automatically.

        # batch_first=True
        self.rnn = torch.nn.RNN(input_size, hidden_size, nonlinearity='relu')
        self.linear = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # The RNN also returns its hidden state but we don't use it.
        # While the RNN can also take a hidden state as input, the RNN
        # gets passed a hidden state initialized with zeros by default.
        print('size of x', x.size())
        h = self.rnn(x)[0]
        print('shape of h', h.shape())
        x = self.linear(h)
        return x

In [54]:
def train(model, train_data_gen, criterion, optimizer, device):
    # Set the model to training mode. This will turn on layers that would
    # otherwise behave differently during evaluation, such as dropout.
    model.train()

    # Store the number of sequences that were classified correctly
    num_correct = 0

    # Iterate over every batch of sequences. Note that the length of a data generator
    # is defined as the number of batches required to produce a total of roughly 1000
    # sequences given a batch size.
    for batch_idx, (data, target) in enumerate(train_data_gen):
        data, target = data.to(device), target.to(device)

        # Perform the forward pass of the model
        output = model(data)  # Step ①

        output = output[:, -1, :]

        # Compute the value of the loss for this batch. For loss functions like CrossEntropyLoss,
        # the second argument is actually expected to be a tensor of class indices rather than
        # one-hot encoded class labels. One approach is to take advantage of the one-hot encoding
        # of the target and call argmax along its second dimension to create a tensor of shape
        # (batch_size) containing the index of the class label that was hot for each sequence.
        target = target.argmax(dim=1)

        loss = criterion(output, target)  # Step ②

        # Clear the gradient buffers of the optimized parameters.
        # Otherwise, gradients from the previous batch would be accumulated.
        optimizer.zero_grad()  # Step ③

        loss.backward()  # Step ④

        optimizer.step()  # Step ⑤

        y_pred = output.argmax(dim=1)
        num_correct += (y_pred == target).sum().item()

    return num_correct, loss.item()

In [55]:
def test(model, test_data_gen, criterion, device):
    # Set the model to evaluation mode. This will turn off layers that would
    # otherwise behave differently during training, such as dropout.
    model.eval()

    # Store the number of sequences that were classified correctly
    num_correct = 0

    # A context manager is used to disable gradient calculations during inference
    # to reduce memory usage, as we typically don't need the gradients at this point.
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_data_gen):

            data, target = data.to(device), target.to(device)

            output = model(data)

            target = target.argmax(dim=1)
            loss = criterion(output, target)

            y_pred = output.argmax(dim=1)
            num_correct += (y_pred == target).sum().item()

    return num_correct, loss.item()

In [56]:
def train_and_test(model, train_data_gen, test_data_gen, criterion, optimizer, max_epochs, verbose=True):
    # Automatically determine the device that PyTorch should use for computation
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Move model to the device which will be used for train and test
    model.to(device)

    # Track the value of the loss function and model accuracy across epochs
    history_train = {'loss': [], 'acc': []}
    history_test = {'loss': [], 'acc': []}

    for epoch in range(max_epochs):
        # Run the training loop and calculate the accuracy.
        # Remember that the length of a data generator is the number of batches,
        # so we multiply it by the batch size to recover the total number of sequences.
        num_correct, loss = train(model, train_data_gen, criterion, optimizer, device)
        accuracy = float(num_correct) / (len(train_data_gen) * train_data_gen.batch_size) * 100
        history_train['loss'].append(loss)
        history_train['acc'].append(accuracy)

        # Do the same for the testing loop
        num_correct, loss = test(model, test_data_gen, criterion, device)
        accuracy = float(num_correct) / (len(test_data_gen) * test_data_gen.batch_size) * 100
        history_test['loss'].append(loss)
        history_test['acc'].append(accuracy)

        if verbose or epoch + 1 == max_epochs:
            print(f'[Epoch {epoch + 1}/{max_epochs}]'
                  f" loss: {history_train['loss'][-1]:.4f}, acc: {history_train['acc'][-1]:2.2f}%"
                  f" - test_loss: {history_test['loss'][-1]:.4f}, test_acc: {history_test['acc'][-1]:2.2f}%")

    # Generate diagnostic plots for the loss and accuracy
    fig, axes = plt.subplots(ncols=2, figsize=(9, 4.5))
    for ax, metric in zip(axes, ['loss', 'acc']):
        ax.plot(history_train[metric])
        ax.plot(history_test[metric])
        ax.set_xlabel('epoch', fontsize=12)
        ax.set_ylabel(metric, fontsize=12)
        ax.legend(['Train', 'Test'], loc='best')
    plt.show()

    return model

In [57]:
# Setup the RNN and training settings

# doing len(train_dataset) gives the number of batches

#rows, columns = train_dataset.get_shape()
input_size  = len(train_loader)
hidden_size = 50
output_size = 4
model       = SimpleRNN(input_size, hidden_size, output_size)
criterion   = torch.nn.CrossEntropyLoss()
optimizer   = torch.optim.RMSprop(model.parameters(), lr=0.001)
max_epochs  = 3

# Train the model
train(model, train_loader, criterion, optimizer, device)
train(model, val_loader, criterion, optimizer, device)


size of x torch.Size([100, 1, 200])


RuntimeError: input.size(-1) must be equal to input_size. Expected 40, got 200